# 1. Importing prepared Data

In [1]:
import pandas as pd
import numpy as np
data = pd.read_excel('data/prepared_data.xlsx')
data.drop(columns= 'Unnamed: 0', inplace= True)
data.rename(columns= {'class' : 'label'}, inplace= True)
data['label'], _ = data['label'].factorize()
data

,text,target,label
0,Жалобы на одышку при умеренной физической нагр...,Z95,0
1,12:50 Жалобы на болезненность при движениях в ...,M17,1
2,"на кашель с трудноотделяемой мокротой, одышка ...",J44,2
3,Осмотр деж врача.\nНа момент осмотра жалобы на...,U07,3
4,Совместный осмотр зав отделением Мурзаевой А.А...,U07,3
...,...,...,...
8167,"Прежние,самочувствие с улучшением .АД стабили...",I11,6
8168,"На головную боль, повышение АД,перебои в работ...",I50,6
8169,"наголовные боли,головокружение , слабость неу...",I11,6
8170,на момент осмотра не предъявляет. Закончились ...,I11,6


In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('prepared1.csv')
data = data.sample(frac=1, random_state= 14).reset_index(drop=True)
data

,text,target,label
0,Жалобы на повышение температуры тела до 384 гр...,U07,4
1,сохраняются жалобы на боль за грудиной после Ф...,Z95,1
2,Осмотр совместно с зав терапевтическим отделен...,U07,4
3,На периодические головные болиповышение АДшум ...,E11,5
4,На боли в эпигастрии и в правом подреберье тош...,K74,6
...,...,...,...
13997,жалобы на слабость кашель с макротой повышени...,J44,3
13998,Жалоб нет Состояние удовлетворительное Кожные ...,Z00,1
13999,на непродуктивный кашель слабость головные бол...,U07,4
14000,на боли в эпигастрии давящего характера тошнот...,K29,6


In [2]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

train_dataset = ds.dataset(pa.Table.from_pandas(data[:10000]).to_batches())
### convert train to Huggingface dataset
hg_train_dataset = Dataset(pa.Table.from_pandas(data[:10000]))


test_dataset = ds.dataset(pa.Table.from_pandas(data[10000:]).to_batches())
### convert test to Huggingface dataset
hg_test_dataset = Dataset(pa.Table.from_pandas(data[10000:]))

# 2. Transformer Model

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny-sentiment-balanced')

In [4]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding = True)

In [5]:
#vectorized = preprocess_function(list(data['text']))
tokenized_train = hg_train_dataset.map(preprocess_function, batched=True, load_from_cache_file=False)
tokenized_test = hg_test_dataset.map(preprocess_function, batched=True, load_from_cache_file=False)
columns_to_return = ['input_ids', 'label', 'attention_mask']
tokenized_train.set_format(type='torch', columns=columns_to_return)
tokenized_test.set_format(type='torch', columns=columns_to_return)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [6]:
tokenized_train['label'][0]

tensor(4)

In [7]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding= True)

In [8]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [9]:
id2label = {0: "Z", 1: "M", 2: "J", 3: "U", 4: "E", 5: "K", 6: "I"}
label2id = {"Z": 0, "M": 1, "J": 2, "U": 3, "E": 4, "K": 5, "I": 6}

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny-sentiment-balanced", num_labels=7, id2label=id2label, label2id=label2id,
    ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny-sentiment-balanced and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 312]) in the checkpoint and torch.Size([7, 312]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    remove_unused_columns=False
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train() # 0.83 ~ 0.84

C:\Users\vnvof\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
  Number of trainable parameters = 11786359
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.464000,0.980657,0.706147
2,0.986500,0.837866,0.750125


***** Running Evaluation *****
  Num examples = 4002
  Batch size = 16
Saving model checkpoint to my_awesome_model\checkpoint-625
Configuration saved in my_awesome_model\checkpoint-625\config.json
Model weights saved in my_awesome_model\checkpoint-625\pytorch_model.bin
tokenizer config file saved in my_awesome_model\checkpoint-625\tokenizer_config.json
Special tokens file saved in my_awesome_model\checkpoint-625\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4002
  Batch size = 16
Saving model checkpoint to my_awesome_model\checkpoint-1250
Configuration saved in my_awesome_model\checkpoint-1250\config.json
Model weights saved in my_awesome_model\checkpoint-1250\pytorch_model.bin
tokenizer config file saved in my_awesome_model\checkpoint-1250\tokenizer_config.json
Special tokens file saved in my_awesome_model\checkpoint-1250\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from my_aw

TrainOutput(global_step=1250, training_loss=1.1613040893554687, metrics={'train_runtime': 280.3788, 'train_samples_per_second': 71.332, 'train_steps_per_second': 4.458, 'total_flos': 147580293120000.0, 'train_loss': 1.1613040893554687, 'epoch': 2.0})

https://huggingface.co/docs/transformers/tasks/sequence_classification